# 🚀 English-Vietnamese Translation Model Training

**Transformer-based Neural Machine Translation from Scratch**

This notebook trains an EN-VI translation model using:
- **PhoMT Dataset** (~3M sentence pairs)
- **SentencePiece Tokenization**
- **Transformer Architecture** with Pre-LayerNorm
- **Beam Search** for inference

---

## 1. ⚙️ Setup Environment

In [ ]:
# Clone repository
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
# Install dependencies (skip torch - already installed on Kaggle)
# Kaggle already has PyTorch with correct CUDA, don't reinstall!
!pip install -q datasets sentencepiece sacrebleu google-generativeai python-dotenv tqdm tensorboard seaborn pyyaml

In [ ]:
# Verify PyTorch and CUDA
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA device: {torch.cuda.get_device_name(0)}')

In [ ]:
# Create .env file with API keys
# ⚠️ IMPORTANT: Replace with your actual keys!

GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"  # Get from Google AI Studio
HF_TOKEN = "YOUR_HF_TOKEN_HERE"  # Get from Hugging Face

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
    f.write(f'HF_TOKEN={HF_TOKEN}\n')

print('✓ .env file created')

## 2. 📥 Download & Preprocess Data

In [ ]:
# Download PhoMT dataset (~500MB)
!python scripts/download_phomt.py

In [ ]:
# Preprocess data: tokenize, filter by length, save to .pt files
# This will train SentencePiece tokenizers and process train/val/test sets
!python scripts/preprocess_data.py

## 3. 🔧 Configuration Check

In [ ]:
import sys
sys.path.insert(0, '.')

from src.config import load_config
from src.models import Transformer

config = load_config()

print("="*50)
print("📋 Configuration Summary")
print("="*50)
print(f"\n🔹 Model:")
print(f"   d_model: {config.d_model}")
print(f"   layers: {config.num_encoder_layers} enc + {config.num_decoder_layers} dec")
print(f"   d_ff: {config.d_ff}")
print(f"   dropout: {config.dropout}")
print(f"\n🔹 Training:")
print(f"   epochs: {config.epochs}")
print(f"   batch_size: {config.batch_size} (effective: {config.batch_size * config.gradient_accumulation_steps})")
print(f"   learning_rate: {config.learning_rate}")
print(f"   warmup_steps: {config.warmup_steps}")
print(f"\n🔹 Data:")
print(f"   vocab_size: {config.src_vocab_size}")
print(f"   max_seq_len: {config.max_seq_len}")
print(f"   data_source: {config.data_source}")

In [ ]:
# Count model parameters
model = Transformer(
    src_vocab_size=config.src_vocab_size,
    tgt_vocab_size=config.tgt_vocab_size,
    d_model=config.d_model,
    num_heads=config.num_heads,
    num_encoder_layers=config.num_encoder_layers,
    num_decoder_layers=config.num_decoder_layers,
    d_ff=config.d_ff
)

total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 Model Parameters: {total_params:,} ({total_params/1e6:.1f}M)")
print(f"📦 Model Size: ~{total_params * 4 / 1024 / 1024:.1f} MB")

## 4. 🏋️ Training

In [ ]:
# Switch to processed data for faster training
import yaml

with open('config/config.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

cfg['data']['source'] = 'processed'  # Use pre-tokenized data

with open('config/config.yaml', 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False)

print('✓ Config updated to use processed data')

In [ ]:
# Start training!
from src.train import Trainer

trainer = Trainer()
trainer.setup()

# Train với hoặc không có checkpoint
print("\n🚀 Starting training from scratch")
trainer.train()